# 1 最长序列问题

最长序列问题是动态规划中，尤其在国内面试中经常考察的问题，主要包括两类：

- 1.最长上升/下降/Wiggle子序列的长度。这类题一般是给定义一个整型数组，来求解数组中的最长的满足某个条件，如上升，子序列的长度。
- 2.最长公共子序列/连续子序列/编辑距离。这类问题一般是给定两个字符串，进行比较求得其公共的序列解。这类问题也是NLP算法面试中经常所考察的问题。

# 2 单一数组的最长序列问题

## 2.1 最长连续子序列

给定一个无序的整数数组，找到其中最长上升子序列的长度。

- 输入: [10,9,2,5,3,7,101,18]
- 输出: 4 
- 解释: 最长的上升子序列是 [2,3,7,101]，它的长度是 4。

这类问题一般很容易找到状态与状态转移，我们定义如下：

- 状态定义：$f(i)$代表以$num[i]$为结尾的最长上升子序列的长度
- 状态转移：$f(i) = f(j) + 1$，其中$j\lt i$且$nums[j] \lt nums[i]$

代码如下：

```java
class Solution {
    public int lengthOfLIS(int[] nums) {
        // 时间复杂度：O(n2)
        // 空间复杂度：O(n)
        
        int len = nums.length;
        if( len == 0 )
            return 0;
        
        int[] dp = new int[len];
        Arrays.fill(dp, 1);
        
        int res = 1;
        for( int i = 1; i < len; i++ ) {
            for( int j = i-1; j >= 0; j-- ) {
                if( nums[j] < nums[i] )
                    dp[i] = Math.max( dp[i], dp[j] + 1 );
            }
            
            res = Math.max( res, dp[i] );
        }
        
        return res;
    }
}
```

## 2.2 最长上升子序列的解

上面的问题我们只是需要求解最长上升子序列的长度，当我们需要求解其中一个最长上升次序列的解时，就需要从dp的结果中进行反推。

具体思路如下：

- 初始化索引$i=len-1$
- 倒序遍历找到最长公共子序列的结尾索引，将对应位置元素纳入res
- 对于当前元素$i$，遍历$j=i-1...0$，当出现第一个$nums[j] \lt nums[i]$且$dp[j] == dp[i]-1$时，更新$i=j$
- 重复上一步操作直到遍历结束，返回res


## 2.3 Wiggle Sequence

题目描述：

如果连续数字之间的差严格地在正数和负数之间交替，则数字序列称为摆动序列。第一个差（如果存在的话）可能是正数或负数。少于两个元素的序列也是摆动序列。

例如， [1,7,4,9,2,5] 是一个摆动序列，因为差值 (6,-3,5,-7,3) 是正负交替出现的。相反, [1,4,7,2,5] 和 [1,7,4,5,5] 不是摆动序列，第一个序列是因为它的前两个差值都是正数，第二个序列是因为它的最后一个差值为零。

给定一个整数序列，返回作为摆动序列的最长子序列的长度。 通过从原始序列中删除一些（也可以不删除）元素来获得子序列，剩下的元素保持其原始顺序。

例如：

- 输入: [1,7,4,9,2,5]
- 输出: 6 
- 解释: 整个序列均为摆动序列。

### 2.3.1 基础解法（时间复杂度n2）

这个题其实是最长上升子序列和最长下降子序列的组合版。由于wiggle中既有上升，又有下降趋势，因此很容易想到用两个状态去表示整个问题的过程：

- 定义状态：$up[i]$代表以$nums[i]$结尾且最后为上升状态的最长wiggle序列；$down[i]$代表以$nums[i]$结尾且最后为下降状态的最长wiggle序列；
- 状态转移：
    - $up[i] = \max\{up[i], down[j] + 1\}$，其中$j\lt i$且$nums[j] \lt nums[i]$
    - $down[i] = \max\{down[i], up[j] + 1\}$，其中$j\lt i$且$nums[j] \gt nums[i]$

代码如下：

```java
class Solution {
    public int wiggleMaxLength(int[] nums) {
        // 时间复杂度：O(n2)
        // 空间复杂度：O(n)
        int len = nums.length;
        if( len == 0 )
            return 0;
        
        int[] up = new int[len];
        int[] down = new int[len];
        Arrays.fill(up, 1);
        Arrays.fill(down, 1);
        
        int res = 1;
        for( int i = 1; i < len; i++ ) {
            for( int j = i-1; j >= 0; j-- ) {
                if( nums[j] < nums[i] ) {
                    up[i] = Math.max( up[i], down[j] + 1 );
                } else if( nums[j] > nums[i] ) {
                    down[i] = Math.max( down[i], up[j] + 1 );
                }
            }
            
            res = Math.max( res, Math.max( up[i], down[i] ));
        }
        
        return res;
    }
}
```

### 2.3.2 优化解法

我们重新定义状态和状态转移，可以对上述问题进行优化：

- 状态定义：$up[i]$代表$[0...i]$区间内最后为上升状态的最长wiggle序列；$down[i]$代表$[0...i]$区间内最后为下降状态的最长
- 状态转移：
    - 如果$nums[i] > nums[i-1]$，则最终状态应该为上升，则$up[i] = down[i-1]+1$
    - 如果$nums[i] < nums[i-1]$，则最终状态应该为下降，则$down[i] = up[i-1]+1$
    - 如果$nums[i] == nums[i-1]$，则$up[i] = up[i-1]$，$down[i] = down[i-1]$

代码如下：

```java
class Solution {
    public int wiggleMaxLength(int[] nums) {
        // 时间复杂度：O(n)
        // 空间复杂度：O(1)
        int len = nums.length;
        if( len == 0 )
            return 0;
        // 空间优化
        int[] up = new int[2];
        int[] down = new int[2];
        Arrays.fill(up, 1);
        Arrays.fill(down, 1);
        
        
        for( int i = 1; i < len; i++ ) {
            if( nums[i] > nums[i-1] ) {
                up[i%2] = Math.max( up[i%2], down[(i-1)%2] + 1 );
                down[i%2] = down[(i-1)%2];
            } else if( nums[i] < nums[i-1] ) {
                down[i%2] = Math.max( down[i%2], up[(i-1)%2] + 1 );
                up[i%2] = up[(i-1)%2];
            } else {
                up[i%2] = up[(i-1)%2];
                down[i%2] = down[(i-1)%2];
            }
        }
        
        return Math.max(up[(len-1)%2], down[(len-1)%2]);
    }
}
```

# 3 字符串的最长公共序列问题

与字符串有关的动态规划问题主要